In [ ]:
%pip install pandas xgboost lbgm

In [5]:
import pandas as pd

df=pd.read_csv('table_fixt.csv', names=['Round Number', 'Location', 'Home', 'Away'])
df.head()

,Round Number,Location,Home,Away
0,NaN,NaN,NaN,NaN
1,Round Number,Location,Home Team,Away Team
2,1,Anfield,7,1
3,1,Villa Park,495,9
4,1,American Express Stadium,2,502


In [6]:
df=df.iloc[2:]

In [7]:
df.head()

,Round Number,Location,Home,Away
2,1,Anfield,7,1
3,1,Villa Park,495,9
4,1,American Express Stadium,2,502
5,1,Stadium of Light,500,14
6,1,Tottenham Hotspur Stadium,15,8


In [9]:
import requests, json
h2h_rec=[]


for index, rows in df.iterrows():
    team1= rows['Home']
    team2= rows['Away']
    url = f"https://livescore-api.com/api-client/teams/head2head.json?team1_id={team1}&team2_id={team2}&key=2O4NK7SsaREzVlj3&secret=PqXIrAwgyGMznJuqyzdiG5KRXprTLKWA"
    # Make the request to the Livescore API
    response = requests.get(url)
    
    data = response.json()
    h2h = data.get("data", [])
    head=h2h.get("team1", [])
    h2h=head.get("h2h_form",[])
    h2h_rec.append(h2h)
df['H2H'] = h2h_rec

In [10]:
score=[]
for index, rows in df.iterrows():
    S=0
    h2h = rows['H2H']
    for match in h2h:
        if match=='W':
            S+=3
        elif match=='L':
            S-=3
        elif match=='D':
            S+=0
    score.append(S)
df['h2h_index']=score
    

In [11]:
df.head()

,Round Number,Location,Home,Away,H2H,h2h_index
2,1,Anfield,7,1,"[W, W, W, W, W, L]",12
3,1,Villa Park,495,9,"[W, L, L, L, D, W]",-3
4,1,American Express Stadium,2,502,"[W, L, L, D, L, L]",-9
5,1,Stadium of Light,500,14,[],0
6,1,Tottenham Hotspur Stadium,15,8,"[W, W, W, W, L, W]",12


In [12]:
df.shape

(380, 6)

In [13]:
elo_df=pd.read_csv('teamid - Sheet1.csv', names=['Team','Team ID', 'Elo Rating'])

In [14]:
elo_df=elo_df[1:]
elo_df.head()

,,Team,Team ID,Elo Rating
Team,team_id,Elo,awin_pct,hwin_pct
Man City,12,1960,42,60
Sunderland,500,1547,18,32
Tottenham,15,1774,33,54
Liverpool,7,1993,43,64


In [15]:
elo_map=dict(zip(elo_df['Team ID'], elo_df['Elo Rating']))

df['homeElo']=df['Home'].map(elo_map)
df['awayElo']=df['Away'].map(elo_map)

In [16]:
df.head()

,Round Number,Location,Home,Away,H2H,h2h_index,homeElo,awayElo
2,1,Anfield,7,1,"[W, W, W, W, W, L]",12,NaN,NaN
3,1,Villa Park,495,9,"[W, L, L, L, D, W]",-3,NaN,NaN
4,1,American Express Stadium,2,502,"[W, L, L, D, L, L]",-9,NaN,NaN
5,1,Stadium of Light,500,14,[],0,NaN,NaN
6,1,Tottenham Hotspur Stadium,15,8,"[W, W, W, W, L, W]",12,NaN,NaN


In [18]:
train_df=pd.read_csv('synth_train_elo_diff_h2h.csv')

In [19]:
train_df.head()

,home_team,away_team,elo_diff,h2h_index,result
0,Arsenal,West Ham,243,1.68,H
1,West Ham,Fulham,-32,2.08,D
2,Fulham,Leeds,60,2.73,D
3,Brentford,Sunderland,264,6.74,H
4,Brentford,Tottenham,37,1.12,D


In [21]:
train_df.shape

(3000, 5)

In [22]:
train_df['result'].value_counts()

result
H    1336
A     873
D     791
Name: count, dtype: int64

In [24]:
df.head()

,Round Number,Location,Home,Away,H2H,h2h_index,homeElo,awayElo
2,1,Anfield,7,1,"[W, W, W, W, W, L]",12,NaN,NaN
3,1,Villa Park,495,9,"[W, L, L, L, D, W]",-3,NaN,NaN
4,1,American Express Stadium,2,502,"[W, L, L, D, L, L]",-9,NaN,NaN
5,1,Stadium of Light,500,14,[],0,NaN,NaN
6,1,Tottenham Hotspur Stadium,15,8,"[W, W, W, W, L, W]",12,NaN,NaN


In [25]:
elo_df=pd.read_csv('teamid - Sheet1.csv', names=['Team','Team ID', 'Elo Rating', 'awayw_pct', 'homew_pct'])

In [26]:
elo_df=elo_df[2:]
elo_df.head()

,Team,Team ID,Elo Rating,awayw_pct,homew_pct
2,Man City,12,1960,42,60
3,Sunderland,500,1547,18,32
4,Tottenham,15,1774,33,54
5,Liverpool,7,1993,43,64
6,Nottm Forest,487,1803,26,37


In [27]:
awayw_pct_map=dict(zip(elo_df['Team ID'], elo_df['awayw_pct']))
homew_pct_map=dict(zip(elo_df['Team ID'], elo_df['homew_pct']))

df['awayW_pct']= df['Away'].map(awayw_pct_map)
df['homeW_pct']= df['Home'].map(homew_pct_map)

In [28]:
df=df.rename(columns={
    'h2h_index': 'h2h_bias',
    'homeElo': 'home_team_elo',
    'awayElo': 'away_team_elo',
    'homeW_pct': 'home_stadium_win_pct',
    'awayW_pct': 'away_stadium_win_pct'
})

In [29]:
df.head()

,Round Number,Location,Home,Away,H2H,h2h_bias,home_team_elo,away_team_elo,away_stadium_win_pct,home_stadium_win_pct
2,1,Anfield,7,1,"[W, W, W, W, W, L]",12,NaN,NaN,26,64
3,1,Villa Park,495,9,"[W, L, L, L, D, W]",-3,NaN,NaN,26,42
4,1,American Express Stadium,2,502,"[W, L, L, D, L, L]",-9,NaN,NaN,18,35
5,1,Stadium of Light,500,14,[],0,NaN,NaN,24,32
6,1,Tottenham Hotspur Stadium,15,8,"[W, W, W, W, L, W]",12,NaN,NaN,20,54


In [30]:
train_df.head()

,home_team,away_team,elo_diff,h2h_index,result
0,Arsenal,West Ham,243,1.68,H
1,West Ham,Fulham,-32,2.08,D
2,Fulham,Leeds,60,2.73,D
3,Brentford,Sunderland,264,6.74,H
4,Brentford,Tottenham,37,1.12,D


In [31]:
df = df[['Home','Away','home_team_elo', 'away_team_elo', 'home_stadium_win_pct', 'away_stadium_win_pct', 'h2h_bias']]

In [32]:
df.head()

,Home,Away,home_team_elo,away_team_elo,home_stadium_win_pct,away_stadium_win_pct,h2h_bias
2,7,1,NaN,NaN,64,26,12
3,495,9,NaN,NaN,42,26,-3
4,2,502,NaN,NaN,35,18,-9
5,500,14,NaN,NaN,32,24,0
6,15,8,NaN,NaN,54,20,12


In [33]:
elo_map=dict(zip(elo_df['Team ID'], elo_df['Elo Rating']))
df['home_team_elo'] = df['Home'].map(elo_map)
df['away_team_elo'] = df['Away'].map(elo_map)

In [34]:
df.head()

,Home,Away,home_team_elo,away_team_elo,home_stadium_win_pct,away_stadium_win_pct,h2h_bias
2,7,1,1993,1808,64,26,12
3,495,9,1873,1869,42,26,-3
4,2,502,1827,1782,35,18,-9
5,500,14,1547,1750,32,24,0
6,15,8,1774,1730,54,20,12


In [36]:

df.to_csv('out/test.csv', index=False)
train_df.to_csv('out/train.csv', index=False)
elo_df.to_csv('out/elo.csv', index=False)



In [69]:
# Features and labels
X = train_df[['elo_dif',
        'home_stadium_win_pct', 'away_stadium_win_pct',
        'h2h_bias']]
y = train_df['result']   # Should be 'H', 'A', or 'D'

In [98]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

y_num=y.map({'H': 0, 'D': 1, 'A': 2})
X_train, X_test, y_train, y_test=train_test_split(X,y_num,test_size=0.3,random_state=42)
cat_clf=CatBoostClassifier(iterations=50, learning_rate=0.05, depth=4, loss_function='MultiClass',
                           eval_metric='Accuracy',class_weights=[1,1.6,1.3],verbose=0, random_seed=42)

cat_clf.fit(X_train, y_train)
y_pred=cat_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
y_proba=cat_clf.predict_proba(X_test)
print(log_loss(y_test, y_proba))


0.4711111111111111
1.0671738269537745


In [103]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, log_loss
from imblearn.over_sampling import SMOTE
import numpy as np

y_num = y.map({'H': 0, 'D': 1, 'A': 2})

# K-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []
logloss_scores = []

for train_idx, test_idx in kf.split(X, y_num):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y_num.iloc[train_idx], y_num.iloc[test_idx]
    
    
    cat_clfk= CatBoostClassifier(
        iterations=50,
        learning_rate=0.05,
        depth=4,
        loss_function='MultiClass',
        eval_metric='Accuracy',
        class_weights=[1, 1.6, 1.3],
        verbose=0,
        random_seed=42
    )
    cat_clfk.fit(X_train, y_train)
    y_pred = cat_clfk.predict(X_test)
    y_proba = cat_clfk.predict_proba(X_test)
    acc_scores.append(accuracy_score(y_test, y_pred))
    logloss_scores.append(log_loss(y_test, y_proba))

print("K-Fold Accuracy: %.4f ± %.4f" % (np.mean(acc_scores), np.std(acc_scores)))
print("K-Fold Log Loss: %.4f ± %.4f" % (np.mean(logloss_scores), np.std(logloss_scores)))

K-Fold Accuracy: 0.4747 ± 0.0110
K-Fold Log Loss: 1.0636 ± 0.0050


In [70]:
df.head()

,Home,Away,home_team_elo,away_team_elo,home_stadium_win_pct,away_stadium_win_pct,h2h_bias,result
2,7,1,1993,1808,64,26,12,0
3,495,9,1873,1869,42,26,-3,0
4,2,502,1827,1782,35,18,-9,0
5,500,14,1547,1750,32,24,0,0
6,15,8,1774,1730,54,20,12,0


In [92]:
df['home_team_elo'] = pd.to_numeric(df['home_team_elo'], errors='coerce')
df['away_team_elo'] = pd.to_numeric(df['away_team_elo'], errors='coerce')
df['elo_dif'] = df['home_team_elo'] - df['away_team_elo']

In [93]:
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

points={
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0

}

X_new=df[['elo_dif','home_stadium_win_pct', 'away_stadium_win_pct', 'h2h_bias']]
y_new=cat_clf.predict(X_new)
y_new=y_new.flatten()  # Convert (n,1) to (n,)

df['result']=y_new


h_ct, a_ct, d_ct=0,0,0
for index, row in df.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']
    

    print(f"Match: {home_team} vs {away_team}, Result: {result}")
    
    if result == 0:
        points[home_team] += 3
        h_ct+=1
    elif result == 2:
        points[away_team] += 3
        a_ct+=1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct+=1

sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
for team, pts in sorted_points.items():
    print(f"{team}:{ pts} points")
print(h_ct, a_ct, d_ct)

Match: Liverpool vs Bournemouth , Result: 0
Match: Aston Villa vs Newcastle Utd, Result: 0
Match: Brighton vs Fulham, Result: 0
Match: Sunderland vs West Ham, Result: 0
Match: Tottenham  vs Burnley, Result: 0
Match: Wolves vs Man City, Result: 2
Match: Nottm Forest vs Brentford, Result: 1
Match: Chelsea vs Crystal Palace, Result: 0
Match: Man Utd vs Arsenal , Result: 2
Match: Leeds vs Everton, Result: 0
Match: West Ham vs Chelsea, Result: 2
Match: Man City vs Tottenham , Result: 0
Match: Bournemouth  vs Wolves, Result: 0
Match: Brentford vs Aston Villa, Result: 0
Match: Burnley vs Sunderland, Result: 0
Match: Arsenal  vs Leeds, Result: 0
Match: Crystal Palace vs Nottm Forest, Result: 0
Match: Everton vs Brighton, Result: 0
Match: Fulham vs Man Utd, Result: 0
Match: Newcastle Utd vs Liverpool, Result: 2
Match: Chelsea vs Fulham, Result: 0
Match: Man Utd vs Burnley, Result: 0
Match: Sunderland vs Brentford, Result: 0
Match: Tottenham  vs Bournemouth , Result: 0
Match: Wolves vs Everton, 

In [96]:
import numpy as np

# Elo update function
def update_elo(elo_home, elo_away, result, k=20):
    """
    Updates Elo ratings after a match.
    result: 'H', 'A', or 'D' (home, away, draw)
    k: K-factor (controls update magnitude)
    """
    # Expected probabilities (logistic model)
    exp_home = 1 / (1 + 10 ** ((elo_away - elo_home) / 400))
    exp_away = 1 - exp_home

    # Actual outcomes
    if result == 0:
        score_home, score_away = 1, 0
    elif result == 2:
        score_home, score_away = 0, 1
    else:  # Draw
        score_home, score_away = 0.5, 0.5

    # Update ratings
    new_home = elo_home + k * (score_home - exp_home)
    new_away = elo_away + k * (score_away - exp_away)

    return new_home, new_away


# ID → Team mapping
id_map = dict(zip(elo_df['Team ID'], elo_df['Team']))

# Initialize points
points = {
    'Liverpool': 0,
    'Man City': 0,
    'Arsenal ': 0,
    'Man Utd': 0,
    'Chelsea': 0,
    'Tottenham ': 0,
    'Aston Villa': 0,
    'Brighton': 0,
    'Newcastle Utd': 0,
    'Brentford': 0,
    'Fulham': 0,
    'Crystal Palace': 0,
    'Wolves': 0,
    'West Ham': 0,
    'Nottm Forest': 0,
    'Bournemouth ': 0,
    'Everton': 0,
    'Sunderland': 0,
    'Burnley': 0,
    'Leeds': 0
}

# Create a dict to hold current Elo ratings (from df)
elos = {}
for _, row in elo_df.iterrows():
    elos[row['Team']] = row['Elo Rating']

# Extract features and predict
X_new = df[['elo_dif', 'home_stadium_win_pct', 'away_stadium_win_pct', 'h2h_bias']]
y_new = cat_clf.predict(X_new).flatten()
df['result'] = y_new

# Counters
h_ct, a_ct, d_ct = 0, 0, 0

# Simulate season
for index, row in df.iterrows():
    home_team = id_map.get(row['Home'], row['Home'])
    away_team = id_map.get(row['Away'], row['Away'])
    result = row['result']

    # Use current Elo values
    elo_home = float(elos[home_team])
    elo_away = float(elos[away_team])

    print(f"Match: {home_team} ({elo_home:.1f}) vs {away_team} ({elo_away:.1f}), Result: {'H' if result == 0 else 'A' if result == 2 else 'D'}")

    # Update points
    if result == 0:
        points[home_team] += 3
        h_ct += 1
    elif result == 2:
        points[away_team] += 3
        a_ct += 1
    elif result == 1:
        points[home_team] += 1
        points[away_team] += 1
        d_ct += 1

    # Update Elo ratings
    new_home, new_away = update_elo(elo_home, elo_away, result, k=20)
    elos[home_team] = new_home
    elos[away_team] = new_away

# Final table
sorted_points = dict(sorted(points.items(), key=lambda item: item[1], reverse=True))
print("\n--- Final Points Table ---")
for team, pts in sorted_points.items():
    print(f"{team}: {pts} points (Elo {elos[team]:.1f})")

print(f"\nHome Wins: {h_ct}, Away Wins: {a_ct}, Draws: {d_ct}")


Match: Liverpool (1993.0) vs Bournemouth  (1808.0), Result: H
Match: Aston Villa (1873.0) vs Newcastle Utd (1869.0), Result: H
Match: Brighton (1827.0) vs Fulham (1782.0), Result: H
Match: Sunderland (1547.0) vs West Ham (1750.0), Result: H
Match: Tottenham  (1774.0) vs Burnley (1730.0), Result: H
Match: Wolves (1736.0) vs Man City (1960.0), Result: A
Match: Nottm Forest (1803.0) vs Brentford (1811.0), Result: D
Match: Chelsea (1903.0) vs Crystal Palace (1835.0), Result: H
Match: Man Utd (1799.0) vs Arsenal  (1993.0), Result: A
Match: Leeds (1722.0) vs Everton (1794.0), Result: H
Match: West Ham (1734.7) vs Chelsea (1911.1), Result: A
Match: Man City (1964.3) vs Tottenham  (1782.7), Result: H
Match: Bournemouth  (1802.9) vs Wolves (1731.7), Result: H
Match: Brentford (1810.8) vs Aston Villa (1882.9), Result: H
Match: Burnley (1721.3) vs Sunderland (1562.3), Result: H
Match: Arsenal  (1997.9) vs Leeds (1734.0), Result: H
Match: Crystal Palace (1826.9) vs Nottm Forest (1803.2), Result: H